# Linear regression

As the response is numerical, we will perform linear regerssion (not logistic). We convert categorical values to numerical for simplicity.

We remove SWEETORSOUR because it is just the categorical equivalent of VALENCE.PLEASANT.

In [29]:
data <- read.csv(file.path("..", "data", "training_data.csv"))
data <- na.omit(data)
data$PL <- data$VALENCE.PLEASANTNESS
data$VALENCE.PLEASANTNESS <- NULL
data$SWEETORSOUR <- NULL
data$Intensity <- as.numeric(data$Intensity)
dim(data)

[1]  708 4872

In [30]:
library(leaps)
library(glmnet)
dim(data)
#library(tidymodels)

[1]  708 4872

As the number of predictors is big (and far greater than the number of observation), we must restrain ourself to the most significant predictors. We can found with regsubset/lasso the best models with at most 30 predictors, than cross-validate to find which model is the best linear model.

In [50]:
idx <- sample(nrow(data), 4*nrow(data)/5)
train.x <- data[idx, -data$PL]
train.y <- data[idx, data$PL]
val.x <- data[-idx, -data$PL]
val.y <- data[-idx, data$PL]

#str(train.data)
#dim(train.data)
cols = names(train.x[, sapply(train.x, function(v) var(v) != 0)])
train.x <- train.x[,cols]
dim(train.x)

[1]  566 2844

In [ ]:
#library(plm)
#idx_remove <- detect.lindep(train.data[-PL])

In [55]:
cv.lasso <- cv.glmnet(as.matrix(train.x), as.matrix(train.y), alpha = 0)
plot(cv.lasso)

ERROR: Error in weighted.mean.default(y, weights): 'x' and 'w' must have the same length


In [40]:
best.lasso <- glmnet(as.matrix(train.data[,1:4872]), train.data$PL, alpha = 1, lambda = cv.lasso$lamda.min)
coef(best.lasso)

ERROR: Error in `[.data.frame`(train.data, , 1:4872): undefined columns selected


In [6]:
?regsubsets

regsubsets {leaps},R Documentation
x,"design matrix or model formula for full model, or biglm object"
data,Optional data frame
y,response vector
weights,weight vector
nbest,number of subsets of each size to record
nvmax,maximum size of subsets to examine
force.in,index to columns of design matrix that should be in all models
force.out,index to columns of design matrix that should be in no models
intercept,Add an intercept?
method,"Use exhaustive search, forward selection, backward selection or sequential replacement to search."


In [20]:
reg.fit <- regsubsets(PL ~ ., train.data, method = 'forward', nvmax = 3)

Warning message in leaps.setup(x, y, wt = wt, nbest = nbest, nvmax = nvmax, force.in = force.in, :
"4550  linear dependencies found"

Reordering variables and trying again:


In [23]:
sum <- summary(reg.fit)$which

NULL